# Concave Delaunay triangulation refinement

This jupyter notebook explains how to use the function _compute\_final\_cell()_ to compute the deformed phase-space cell with conserved volume. This makes it possible to parallelize the code in the way that suits the computing system. 

## Computation of the Delaunay triangulation.

Before using any function to compute the deformed cell. The Delaunay triangulation of the initial frame must be computed. \
First the YAML file must be filled. \
Then run the python script CreateTri.py. It will create two pickle objects. One being the Delaunay triangulation (from scipy) and the other the neighbouring list of each point of the triangulation (computed and save to optimize computation time). \
Once the triangulation and neighbour’s list are created the phase space cell with conserved volume can be computed. The notebook show how to do it.

### 1) download the neccessary library 

In [1]:
import numpy as np
from scipy.spatial import Delaunay
import matplotlib.pyplot as plt
from pNbody import*
import pickle
import yaml
import matplotlib
matplotlib.rcParams.update({'font.size': 14})

from concaveDelaunayRefinement import*
from toolMockImage import*

--------------------------------------------------------------------------
but there are no active ports detected (or Open MPI was unable to use
them).  This is most certainly not what you wanted.  Check your
cables, subnet manager configuration, etc.  The openib BTL will be
ignored for this job.

  Local host: lesta02
--------------------------------------------------------------------------


## 2) Download the information from the YAML file and the pickle object

In [2]:
with open("paraImage.yaml") as stream:
    try:
        para = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

ImageFrame = para["SimulationName"]
trans = para["translation"]
triName = para["triSaveName"]
NeighborsName = para["NeighborsSaveName"]
InitialFrame = para["InitialFrameName"]
Nmin = para["Nmin"]
Rmax = para["Rmax"]
halfBoxSize = para["halfBoxSize"]


file_path = NeighborsName
with open(file_path, 'rb') as file:
    loaded_data = pickle.load(file)

print("The variable 'Neighbors' has been loaded successfully.")

neighbors = loaded_data

file_path = triName
with open(file_path, 'rb') as file:
    loaded_data = pickle.load(file)

print("The variable 'Tri' has been loaded successfully.")

tri0 = loaded_data

The variable 'Neighbors' has been loaded successfully.
The variable 'Tri' has been loaded successfully.


## 3) Download the initial frame and the image frame. Compute and save the scaling

The function get_nbScaledList return a list [Nbody object of initial frame (scaled), Nbody object of image frame (scaled)] (Nbody object of pNbody) and the list of standard deviation before scaling [$\sigma_x, \sigma_y,\sigma_z,\sigma_{v_x},\sigma_{v_y},\sigma_{v_z}$]. This function is made to give the necessary variable of _compute\_final\_cell()_.

In [3]:
nb_scaled_tList,std_list = get_nbScaledList(InitialFrame,ImageFrame,trans)

## 4) compute the cell with conserved volume

The function _compute\_final\_cell()_ return the deformed phase-space cell. The cell is defined and its volume computed in the initial frame. Then the cell is reconstructed in the image frame by forcing phase space volume conservation. The function returns the flag, 0 if the cell is correctly reconstructed, 1 otherwise and a list of 6 dimensional triangle. The cell is constituted of those 6 Dimensional triangles. \
\
_compute\_final\_cell()_ take as the first argument the id of the particle. The id of a particle can be found using pNbody nb.num. The Nbody object are found in nb_scaled_tList. A good practice will be to define variable nb0 = nb_scaled_tList[0]. \
Then to apply the function _compute\_final\_cell()_ on all particles its need to run a loop _for id in nb0.num_. Realistically this loop takes too much time (more than 10 days for 100'000 particles). The code must be parallelized. The way someone does parallelization be let to the user. (One parallelization (and cell projection) is made with PartialImage.py).\
Once all the deformed cell has been computed they need to be projected. One way to project they are made in PartialImage.py and BuildImage.py but they are not the only (or optimal for other computing systems) way to do the projection (and they need a DirectX11 environment). 

In [4]:
flag,triangle6D_list = compute_final_cell(nb_scaled_tList[0].num[50],nb_scaled_tList,neighbors,Rmax,Nmin,std_list,verbos=True)

DEBBUG :: cell creation
flag=0
DEBUG : Concave Delaunay Algorithm
DEBUG :: Volume_evolution_Liouville_condition_concave_delaunay_refinements_test :: Creation des triangulation de Delaunay
DEBUG :: Volume_evolution_Liouville_condition_concave_delaunay_refinements_test :: Calcule du phase-space volume
DEBUG :: Volume_evolution_Liouville_condition_concave_delaunay_refinements_test :: phase-space volume V = 1174.8541633360267
Calcul du concave delaunay frame 1
DV/V = 2.6619440430856893%
flag=0
